此处用于聚合所有模块进行训练
首先两两组合

In [ ]:
# train_with_eca_csab.py
from ultralytics import YOLO
import torch.nn as nn
from eca_module import ECA
from csab_offset import CSAB_Offset

# === Step 1: 加载我们已有的最优基线模型 ===
# 确保所有实验都从同一起点开始，保证公平性。
print("Loading baseline model: yolo11n_final4.pt")
model = YOLO("yolo11n_final4.pt")
print("Model loaded successfully.")

# === Step 2: 定义一个函数来同时注入两个模块 ===
def inject_eca_and_csab(model, eca_layers=(4, 7, 10), csab_layers=(8, 10), eca_k_size=3):
    """
    一个函数内完成ECA和CSAB_Offset模块的注入。
    - ECA 模块将被包裹在原始层外部。
    - CSAB_Offset 模块也将以类似方式注入，增强空间特征提取。
    """
    print("Starting module injection process...")
    
    # 注入 ECA 模块
    for i in eca_layers:
        try:
            old_layer = model.model[i]
            # 获取输出通道数
            if hasattr(old_layer, 'cv2'):
                ch = old_layer.cv2.conv.out_channels
            elif isinstance(old_layer, nn.Sequential) and hasattr(list(old_layer.children())[-1], 'out_channels'):
                 ch = list(old_layer.children())[-1].out_channels
            else:
                 # 尝试从常见的属性获取通道数
                 ch = old_layer.out_channels if hasattr(old_layer, 'out_channels') else old_layer.conv.out_channels
            
            # 使用nn.Sequential将原始层和ECA模块包裹起来
            model.model[i] = nn.Sequential(old_layer, ECA(ch, eca_k_size))
            print(f"✅ Injected ECA after layer {i}, channels={ch}")
        except Exception as e:
            print(f"❌ Failed to inject ECA at layer {i}: {e}")

    # 注入 CSAB_Offset 模块
    # 注意：这里的注入逻辑需要根据您的模型结构精确调整
    # 一个常见的做法是替换掉某个关键的 block 或者 Conv 层
    for i in csab_layers:
        try:
            old_layer = model.model[i]
            # 同样地，获取通道数
            if hasattr(old_layer, 'cv2'):
                 ch = old_layer.cv2.conv.out_channels
            else:
                 ch = old_layer.out_channels if hasattr(old_layer, 'out_channels') else old_layer.conv.out_channels
            
            # 这里我们用CSAB_Offset替换原始层
            model.model[i] = CSAB_Offset(ch) 
            print(f"✅ Replaced layer {i} with CSAB_Offset, channels={ch}")
        except Exception as e:
            print(f"❌ Failed to inject CSAB at layer {i}: {e}")

    print("Module injection complete.")
    return model

# === Step 3: 注入模块并验证模型结构 ===
# 我们选择在骨干网络深层的关键block后插入模块，这些地方的特征图语义信息更丰富
model = inject_eca_and_csab(model, eca_layers=[6, 9], csab_layers=[10, 13])

# === Step 4: 保存注入了双模块的结构版模型 ===
# 这个权重是随机初始化的（对于新模块），需要Finetune
init_weights_path = "weights/yolov11n_eca_csab_init.pt"
model.save(init_weights_path)
print(f"Initial model with ECA and CSAB modules saved to {init_weights_path}")

# === Step 5: 在ECA+CSAB版本上进行Finetune训练 ===
# 使用与单模块实验一致的超参数以进行公平比较
print("Starting Finetune training for ECA + CSAB model...")
results = model.train(
    data="pest24.yaml",
    seed=42,
    epochs=50,
    batch=24,
    workers=2,
    lr0=1e-3,                
    lrf=0.05,                 
    optimizer="AdamW",
    weight_decay=0.001,
    dropout=0.2,
    cos_lr=True,              # 使用余弦退火学习率
    cache=True,
    amp=True,
    device=0,
    patience=15,              # 耐心值
    save_period=10,
    name='finetune_eca_csab'   # 清晰的实验命名
)

# === Step 6: 保存最终Finetune完成的权重 ===
final_weights_path = "weights/yolov11n_eca_csab_finetuned.pt"
model.save(final_weights_path)
print(f"Final finetuned model with ECA and CSAB saved to {final_weights_path}")

In [ ]:
# train_with_eca_dbl.py
from ultralytics import YOLO
import torch.nn as nn
from eca_module import ECA
from dbl_loss_tsd import DBL_TSD

# === Step 1: 加载我们已有的最优基线模型 ===
print("Loading baseline model: yolo11n_final4.pt")
model = YOLO("yolo11n_final4.pt")
print("Model loaded successfully.")

# === Step 2: 注入 ECA 模块 ===
# 沿用之前的注入逻辑
def inject_eca(model, layers=(4, 7, 10), k_size=3):
    print("Starting ECA module injection...")
    for i in layers:
        try:
            old_layer = model.model[i]
            # 智能获取输出通道数
            ch = -1
            if hasattr(old_layer, 'cv2') and hasattr(old_layer.cv2, 'conv'):
                ch = old_layer.cv2.conv.out_channels
            elif isinstance(old_layer, nn.Sequential) and hasattr(list(old_layer.children())[-1], 'out_channels'):
                 ch = list(old_layer.children())[-1].out_channels
            else: # 备用方案
                 ch = old_layer.out_channels if hasattr(old_layer, 'out_channels') else old_layer.conv.out_channels
            
            model.model[i] = nn.Sequential(old_layer, ECA(ch, k_size))
            print(f"✅ Injected ECA after layer {i}, channels={ch}")
        except Exception as e:
            print(f"❌ Failed to inject ECA at layer {i}: {e}")
    print("ECA injection complete.")
    return model

model = inject_eca(model, layers=[6, 9]) # 在骨干网深层注入

# === Step 3: 定义并注册 DBL_TSD 损失函数回调 ===
print("Setting up DBL_TSD loss callback...")
loss_wrapper = DBL_TSD(cls_w=1.0, box_w=5.0, iou_w=2.0, scale_alpha=1.0, trunc_thresh=0.4)

def custom_loss_callback(trainer):
    # 注意: trainer对象的属性可能因版本而异，这里基于您的代码结构
    # 您可能需要调试并确认'preds'和'targets'的正确获取方式
    try:
        preds = trainer.preds # Ultralytics v8+ 通常在这里
        targets = trainer.batch
        loss_val, info = loss_wrapper(preds, targets)
        trainer.loss = loss_val
    except Exception as e:
        # 如果上述方式失败，这是一个备用方案，但可能不完全准确
        # print(f"Callback Warning: Could not apply custom loss via standard attributes. Error: {e}")
        pass # 保持默认损失计算
    return trainer.loss

model.add_callback("on_train_batch_end", custom_loss_callback)
print("✅ DBL_TSD loss callback registered.")


# === Step 4: 保存注入了双模块的结构版模型 ===
init_weights_path = "weights/yolov11n_eca_dbl_init.pt"
model.save(init_weights_path)
print(f"Initial model with ECA and DBL_TSD setup saved to {init_weights_path}")

# === Step 5: 在 ECA+DBL_TSD 版本上进行Finetune训练 ===
print("Starting Finetune training for ECA + DBL_TSD model...")
results = model.train(
    data="pest24.yaml",
    seed=42,
    epochs=50,
    batch=24,
    workers=2,
    lr0=1e-3,                
    lrf=0.05,                 
    optimizer="AdamW",
    weight_decay=0.001,
    dropout=0.2,
    cos_lr=True,
    cache=True,
    amp=True,
    device=0,
    patience=15,
    save_period=10,
    name='finetune_eca_dbl'   # 清晰的实验命名
)

# === Step 6: 保存最终Finetune完成的权重 ===
final_weights_path = "weights/yolov11n_eca_dbl_finetuned.pt"
model.save(final_weights_path)
print(f"Final finetuned model with ECA and DBL_TSD saved to {final_weights_path}")

In [ ]:
# train_with_csab_dbl.py
from ultralytics import YOLO
import torch.nn as nn
from csab_offset import CSAB_Offset
from dbl_loss_tsd import DBL_TSD

# === Step 1: 加载我们已有的最优基线模型 ===
print("Loading baseline model: yolo11n_final4.pt")
model = YOLO("yolo11n_final4.pt")
print("Model loaded successfully.")

# === Step 2: 注入 CSAB_Offset 模块 ===
def inject_csab_offset(model, layers=(10, 13)):
    print("Starting CSAB_Offset module injection...")
    for i in layers:
        try:
            old_layer = model.model[i]
            # 获取输出通道数
            ch = -1
            if hasattr(old_layer, 'cv2') and hasattr(old_layer.cv2, 'conv'):
                 ch = old_layer.cv2.conv.out_channels
            else:
                 ch = old_layer.out_channels if hasattr(old_layer, 'out_channels') else old_layer.conv.out_channels
            
            # 使用CSAB_Offset模块替换原有的层
            model.model[i] = CSAB_Offset(ch) 
            print(f"✅ Replaced layer {i} with CSAB_Offset, channels={ch}")
        except Exception as e:
            print(f"❌ Failed to inject CSAB at layer {i}: {e}")
    print("CSAB_Offset injection complete.")
    return model

model = inject_csab_offset(model, layers=[10, 13])

# === Step 3: 定义并注册 DBL_TSD 损失函数回调 ===
print("Setting up DBL_TSD loss callback...")
loss_wrapper = DBL_TSD(cls_w=1.0, box_w=5.0, iou_w=2.0, scale_alpha=1.0, trunc_thresh=0.4)

def custom_loss_callback(trainer):
    try:
        preds = trainer.preds
        targets = trainer.batch
        loss_val, info = loss_wrapper(preds, targets)
        trainer.loss = loss_val
    except Exception as e:
        # print(f"Callback Warning: Could not apply custom loss. Error: {e}")
        pass
    return trainer.loss

model.add_callback("on_train_batch_end", custom_loss_callback)
print("✅ DBL_TSD loss callback registered.")

# === Step 4: 保存注入了双模块的结构版模型 ===
init_weights_path = "weights/yolov11n_csab_dbl_init.pt"
model.save(init_weights_path)
print(f"Initial model with CSAB_Offset and DBL_TSD setup saved to {init_weights_path}")

# === Step 5: 在 CSAB+DBL_TSD 版本上进行Finetune训练 ===
print("Starting Finetune training for CSAB_Offset + DBL_TSD model...")
results = model.train(
    data="pest24.yaml",
    seed=42,
    epochs=50,
    batch=24,
    workers=2,
    lr0=1e-3,                
    lrf=0.05,                 
    optimizer="AdamW",
    weight_decay=0.001,
    dropout=0.2,
    cos_lr=True,
    cache=True,
    amp=True,
    device=0,
    patience=15,
    save_period=10,
    name='finetune_csab_dbl'   # 清晰的实验命名
)

# === Step 6: 保存最终Finetune完成的权重 ===
final_weights_path = "weights/yolov11n_csab_dbl_finetuned.pt"
model.save(final_weights_path)
print(f"Final finetuned model with CSAB_Offset and DBL_TSD saved to {final_weights_path}")

In [ ]:
# train_with_all_modules.py
from ultralytics import YOLO
import torch.nn as nn
from eca_module import ECA
from csab_offset import CSAB_Offset
from dbl_loss_tsd import DBL_TSD

# === Step 1: 加载我们已有的最优基线模型 ===
print("Loading baseline model: yolo11n_final4.pt")
model = YOLO("yolo11n_final4.pt")
print("Model loaded successfully.")

# === Step 2: 定义一个函数来同时注入所有模型架构模块 ===
def inject_all_modules(model, eca_layers=(6, 9), csab_layers=(10, 13)):
    """在一个函数内完成ECA和CSAB_Offset模块的注入。"""
    print("Starting module injection process for ECA and CSAB_Offset...")
    
    # 注入 ECA 模块
    for i in eca_layers:
        try:
            old_layer = model.model[i]
            ch = old_layer.cv2.conv.out_channels if hasattr(old_layer, 'cv2') else old_layer.conv.out_channels
            model.model[i] = nn.Sequential(old_layer, ECA(ch))
            print(f"✅ Injected ECA after layer {i}, channels={ch}")
        except Exception as e:
            print(f"❌ Failed to inject ECA at layer {i}: {e}")

    # 注入 CSAB_Offset 模块
    for i in csab_layers:
        try:
            old_layer = model.model[i]
            ch = old_layer.cv2.conv.out_channels if hasattr(old_layer, 'cv2') else old_layer.conv.out_channels
            model.model[i] = CSAB_Offset(ch) 
            print(f"✅ Replaced layer {i} with CSAB_Offset, channels={ch}")
        except Exception as e:
            print(f"❌ Failed to inject CSAB at layer {i}: {e}")

    print("Model architecture module injection complete.")
    return model

# === Step 3: 注入模型架构模块 ===
model = inject_all_modules(model, eca_layers=[6, 9], csab_layers=[10, 13])

# === Step 4: 定义并注册 DBL_TSD 损失函数回调 ===
print("Setting up DBL_TSD loss callback...")
loss_wrapper = DBL_TSD(cls_w=1.0, box_w=5.0, iou_w=2.0, scale_alpha=1.0, trunc_thresh=0.4)

def custom_loss_callback(trainer):
    try:
        preds = trainer.preds
        targets = trainer.batch
        loss_val, info = loss_wrapper(preds, targets)
        trainer.loss = loss_val
    except Exception as e:
        # print(f"Callback Warning: Could not apply custom loss. Error: {e}")
        pass
    return trainer.loss

model.add_callback("on_train_batch_end", custom_loss_callback)
print("✅ DBL_TSD loss callback registered.")


# === Step 5: 保存注入了所有模块的结构版模型 ===
init_weights_path = "weights/yolov11n_eca_csab_dbl_init.pt"
model.save(init_weights_path)
print(f"Initial model with all modules setup saved to {init_weights_path}")

# === Step 6: 在“三合一”版本上进行Finetune训练 ===
print("Starting Finetune training for the ultimate ECA + CSAB_Offset + DBL_TSD model...")
results = model.train(
    data="pest24.yaml",
    seed=42,
    epochs=50,
    batch=24,
    workers=2,
    lr0=1e-3,                
    lrf=0.05,                 
    optimizer="AdamW",
    weight_decay=0.001,
    dropout=0.2,
    cos_lr=True,
    cache=True,
    amp=True,
    device=0,
    patience=15,
    save_period=10,
    name='finetune_eca_csab_dbl'   # 清晰的实验命名
)

# === Step 7: 保存最终Finetune完成的权重 ===
final_weights_path = "weights/yolov11n_eca_csab_dbl_finetuned.pt"
model.save(final_weights_path)
print(f"Final finetuned model with all modules saved to {final_weights_path}")